<h1><a>Lec 02 NLP基础与扫盲</a></h1>

本节以NLTK为基础配合讲解自然语言处理的原理。

## 目录：
- <a href='#nltk'>NLTK</a>
- <a href='#textprocessing'>文本处理流程</a>
    - <a href='#tokenize'>分词</a>
    - <a href='#standard'>归一化</a>
    - <a href='#stopwords'>停止词</a>
- <a href='#3examples'>NLP经典三案例</a>
    - <a href='#sentiment'>情感分析</a>
    - <a href='#similarity'>文本相似度</a>
    - <a href='#classification'>文本分类</a>
- <a href='#deeplearning'>深度学习加持</a>
    - <a herf='#autoencoder'>Autoencoder</a>
    - <a herf='#word2vec'>Word2Vec</a>